# Clinical data cleaning

In [3]:
import pandas as pd

# Load the already-filtered file
file_path = r"C:\Users\DELL\Downloads\clinvar_result.txt"
df = pd.read_csv(file_path, sep='\t', low_memory=False)

In [37]:
df.head()

,Name,Gene(s),Protein change,Condition(s),Accession,GRCh37Chromosome,GRCh37Location,GRCh38Chromosome,GRCh38Location,VariationID,...,Germline classification,Germline date last evaluated,Germline review status,Somatic clinical impact,Somatic clinical impact date last evaluated,Somatic clinical impact review status,Oncogenicity classification,Oncogenicity date last evaluated,Oncogenicity review status,Unnamed: 24
0,NM_000492.4(CFTR):c.1A>G (p.Met1Val),CFTR,M1V,Cystic fibrosis,VCV000053423,7,117120149,7,117480095,53423,...,Pathogenic,"Mar 17, 2017",reviewed by expert panel,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NM_000492.4(CFTR):c.4C>T (p.Gln2Ter),CFTR,Q2*,Cystic fibrosis,VCV000053980,7,117120152,7,117480098,53980,...,Pathogenic,"Mar 17, 2017",reviewed by expert panel,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NM_000492.4(CFTR):c.11C>A (p.Ser4Ter),CFTR,S4*,Cystic fibrosis,VCV000053211,7,117120159,7,117480105,53211,...,Pathogenic,"Mar 17, 2017",reviewed by expert panel,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NM_000492.4(CFTR):c.38C>T (p.Ser13Phe),CFTR,S13F,Cystic fibrosis,VCV000053845,7,117120186,7,117480132,53845,...,Pathogenic,"Aug 31, 2018",reviewed by expert panel,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NM_000492.4(CFTR):c.44T>C (p.Leu15Pro),CFTR,L15P,Cystic fibrosis,VCV000634833,7,117120192,7,117480138,634833,...,Pathogenic,"Aug 31, 2018",reviewed by expert panel,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:

# Step 1: Confirm CFTR is present
print("Top Genes:")
df['Gene(s)'].value_counts().head(10)

Top Genes:


Gene(s)
CFTR    226
Name: count, dtype: int64

In [5]:

# Step 2: Filter for CFTR gene
df_cftr = df[df['Gene(s)'] == 'CFTR']

In [6]:

# Step 3: See what conditions exist
print(" Top conditions for CFTR:")
df_cftr['Condition(s)'].value_counts().head(15)

 Top conditions for CFTR:


Condition(s)
Cystic fibrosis                                  214
ivacaftor response - Efficacy|Cystic fibrosis      6
Cystic fibrosis|ivacaftor response - Efficacy      6
Name: count, dtype: int64

In [7]:

# Just keep useful columns
columns_to_keep = [
    'Name',
    'Gene(s)',
    'Protein change',
    'Variant type',
    'Molecular consequence',
    'Germline classification',
    'Germline review status',
    'GRCh38Chromosome',
    'GRCh38Location'
]

In [8]:

# Clean data
df_clean = df_cftr[columns_to_keep].drop_duplicates()

In [9]:
# Save clean file
df_clean.to_csv("cftr_mutations_clean.csv", index=False)
print(" Cleaned file saved. Preview:")
df_clean.tail()


 Cleaned file saved. Preview:


,Name,Gene(s),Protein change,Variant type,Molecular consequence,Germline classification,Germline review status,GRCh38Chromosome,GRCh38Location
221,NM_000492.4(CFTR):c.4144C>T (p.Gln1382Ter),CFTR,Q1382*,single nucleotide variant,nonsense,Pathogenic,reviewed by expert panel,7,117665466
222,NM_000492.4(CFTR):c.4231C>T (p.Gln1411Ter),CFTR,Q1411*,single nucleotide variant,nonsense,Pathogenic,reviewed by expert panel,7,117665553
223,NM_000492.4(CFTR):c.4234C>T (p.Gln1412Ter),CFTR,Q1412*,single nucleotide variant,nonsense,Pathogenic,reviewed by expert panel,7,117665556
224,NM_000492.4(CFTR):c.4242+1G>T,CFTR,NaN,single nucleotide variant,splice donor variant,Pathogenic,reviewed by expert panel,7,117665565
225,NM_000492.4(CFTR):c.4242+1G>A,CFTR,NaN,single nucleotide variant,splice donor variant,Pathogenic,reviewed by expert panel,7,117665565


#  Mutation Statistics Section

In [10]:
# Mutation summary
print("\n Mutation Type Summary:")
print(df_clean['Variant type'].value_counts())

print("\n Top Molecular Consequences:")
print(df_clean['Molecular consequence'].value_counts())

print("\n Top Protein Changes:")
df_clean['Protein change'].value_counts().head(5)



 Mutation Type Summary:
Variant type
single nucleotide variant    154
Deletion                      48
Duplication                   18
Indel                          4
Insertion                      2
Name: count, dtype: int64

 Top Molecular Consequences:
Molecular consequence
nonsense                                    69
missense variant                            67
frameshift variant                          63
splice donor variant                        16
splice acceptor variant                     10
missense variant|initiator_codon_variant     1
Name: count, dtype: int64

 Top Protein Changes:


Protein change
W1204*    2
Y913*     2
Q685fs    2
G628R     2
W401*     2
Name: count, dtype: int64

In [11]:
summary = df_clean.groupby(['Variant type', 'Molecular consequence','Protein change']).size().reset_index(name='Count')
summary.to_csv("cftr_mutation_summary.csv", index=False)
summary

,Variant type,Molecular consequence,Protein change,Count
0,Deletion,frameshift variant,A959fs,1
1,Deletion,frameshift variant,D1201fs,1
2,Deletion,frameshift variant,D1202fs,1
3,Deletion,frameshift variant,D648fs,1
4,Deletion,frameshift variant,F143fs,1
...,...,...,...,...
177,single nucleotide variant,nonsense,W882*,1
178,single nucleotide variant,nonsense,Y122*,1
179,single nucleotide variant,nonsense,Y275*,1
180,single nucleotide variant,nonsense,Y849*,1


In [17]:
# Load cleaned CSV
df = pd.read_csv("cftr_mutations_clean.csv")

# Extract positions from 'Protein change' (like p.Phe508del → 508)
df['Position'] = df['Protein change'].str.extract(r'(\d+)')

# Drop null positions
df_pos = df.dropna(subset=['Position'])

# Save list of positions to use in Chimera
positions = df_pos['Position'].unique().astype(int).tolist()
positions.sort()

print("Unique mutation residue positions in CFTR:", positions[:], "...")


Unique mutation residue positions in CFTR: [1, 2, 4, 13, 15, 17, 19, 27, 30, 39, 46, 67, 75, 79, 88, 91, 92, 98, 99, 102, 104, 109, 110, 116, 117, 122, 126, 137, 139, 142, 143, 148, 161, 165, 177, 178, 183, 191, 192, 193, 194, 199, 205, 206, 216, 218, 220, 227, 232, 254, 268, 275, 276, 284, 287, 311, 330, 334, 336, 338, 341, 346, 347, 352, 361, 378, 379, 401, 560, 561, 563, 568, 569, 574, 585, 598, 601, 609, 613, 628, 637, 641, 648, 656, 663, 664, 673, 684, 685, 709, 710, 715, 720, 726, 732, 739, 745, 748, 764, 785, 792, 818, 821, 822, 831, 835, 846, 849, 851, 861, 864, 868, 882, 890, 912, 913, 922, 927, 938, 941, 942, 945, 953, 959, 966, 970, 979, 1128, 1145, 1158, 1159, 1179, 1201, 1202, 1204, 1231, 1234, 1240, 1244, 1249, 1250, 1251, 1254, 1255, 1258, 1269, 1274, 1283, 1295, 1297, 1298, 1301, 1303, 1310, 1313, 1324, 1330, 1335, 1349, 1360, 1363, 1366, 1371, 1375, 1376, 1382, 1383, 1411, 1412] ...


In [18]:
# Group by position and classification

summary = df.groupby(['Position', 'Germline classification']).size().reset_index(name='Count')
summary.to_csv("cftr_classification_by_position.csv", index=False)
summary

,Position,Germline classification,Count
0,1,Pathogenic,1
1,102,Pathogenic,1
2,104,Pathogenic,1
3,109,Pathogenic,1
4,110,Pathogenic; drug response,1
...,...,...,...
167,966,Pathogenic,1
168,970,Pathogenic,2
169,979,Pathogenic,1
170,98,Pathogenic,2
